In [1]:
from DataLoader import importar_ventas,importar_cluster_stores,leer_csv,agregar_fourier,preparar_test
from DataLoader import generar_rolling_features,crear_archivo_kaggle
import importlib
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import r2_score
import numpy as np

In [2]:
def r2_score_lgb(y_true, y_pred):
    return 'r2', r2_score(y_true, y_pred), True 

In [3]:
ventas=importar_ventas()

In [4]:
agregar_fourier(ventas,2)

In [5]:
ventas=generar_rolling_features(ventas,"mean_price",[1,2],["subgroup_cod","store_cod"])
ventas=generar_rolling_features(ventas,"demand",[1,2],["subgroup_cod","store_cod"])

In [6]:
ventas.columns

Index(['date', 'demand', 'mean_price', 'std_price', 'max_price', 'min_price',
       'mean_discount', 'mean_diff_factory', 'std_discount', 'max_discount',
       'min_discount', 'std_diff_factory', 'max_diff_factory',
       'min_diff_factory', 'size', 'group', 'category', 'cluster', 'store_cod',
       'subgroup_cod', 'day', 'month', 'year', 'anual_sin_1', 'anual_cos_1',
       'mensual_sin_1', 'mensual_cos_1', 'anual_sin_2', 'anual_cos_2',
       'mensual_sin_2', 'mensual_cos_2', 'mean_price_rolling_mean_7d',
       'mean_price_rolling_sum_7d', 'mean_price_rolling_mean_14d',
       'mean_price_rolling_sum_14d', 'demand_rolling_mean_7d',
       'demand_rolling_sum_7d', 'demand_rolling_mean_14d',
       'demand_rolling_sum_14d'],
      dtype='object')

In [7]:
FEATURES=[
   #'mean_price', 
    'cluster', 
    #'store_cod', 
    #'subgroup_cod',
    #'demand_lag_1', #'demand_lag_2', 'demand_lag_3', 
    #'demand_lag_4','demand_lag_5', 'demand_lag_6', 'demand_lag_7', 'day', 'month', 
    'year',
   'anual_sin_1', 
    'anual_cos_1',
   'mensual_sin_1', 
    'mensual_cos_1',
   #'anual_sin_2', 
    #'anual_cos_2',
   # 'mensual_sin_2', 
    #'mensual_cos_2',
   # "std_price",
    "max_price", 
   "min_price",
    #'mean_price_rolling_mean_7d',
   #    'mean_price_rolling_sum_7d',# 'mean_price_rolling_mean_14d',
       #'mean_price_rolling_sum_14d', 
   # 'mean_price_rolling_mean_21d',
     #  'mean_price_rolling_sum_21d', 'mean_price_rolling_mean_28d',
    #  'mean_price_rolling_sum_28d', 
    #'demand_rolling_mean_7d',
    #  'demand_rolling_sum_7d', 
    'demand_rolling_mean_14d',
     #  'demand_rolling_sum_14d', #'demand_rolling_mean_21d',
       #'demand_rolling_sum_21d', #'demand_rolling_mean_28d',
      # 'demand_rolling_sum_28d'
    'std_discount', 'max_discount', 'min_discount', 'std_diff_factory',
    'max_diff_factory', 'min_diff_factory'
] #recordar que ya probamos con fourier 3 y 4 y fue un fracaso

test=preparar_test("ids_test.csv",ventas,FEATURES)
TARGET=['demand']

test_interno=ventas.groupby(["subgroup_cod","store_cod"]).tail(7)
train=ventas
train_interno=ventas.drop(test_interno.index)

X_train=train[FEATURES]
y_train=train[TARGET]
X_test_real=test[FEATURES]

X_train_interno=train_interno[FEATURES]
y_train_interno=train_interno[TARGET]

X_test_interno=test_interno[FEATURES]
y_test_interno=test_interno[TARGET]

# X_test_2023=test_2023[FEATURES]
# y_test_2023=test_2023[TARGET]

# X_train_2023=train_2023[FEATURES]
# y_train_2023=train_2023[TARGET]

D:\ESTUDIO\proyecto_eci\DataLoader.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ventas["date"]=pd.to_datetime(ventas["date"])


In [8]:

"""   
# Parameters
initial_train_weeks = 146 
horizon_weeks = 1           
early_stopping_rounds = 50  

# Store results
all_scores = []

# Pre-calculate week-year identifiers (optimized)
ventas['week_year'] = (ventas['date'].dt.isocalendar().year * 100 + 
                       ventas['date'].dt.isocalendar().week)
all_weeks = sorted(ventas['week_year'].unique())

# Ensure data is sorted by date to prevent leakage
ventas = ventas.sort_values('date')

# Rolling window validation
for i in range(initial_train_weeks, len(all_weeks) - horizon_weeks + 1):
    print(i)
    train_weeks = all_weeks[:i]
    val_weeks = all_weeks[i:i + horizon_weeks]
    
    # Create masks using the pre-calculated week_year
    train_mask = ventas['week_year'].isin(train_weeks)
    val_mask = ventas['week_year'].isin(val_weeks)
    
    # Data splitting
    X_train = ventas.loc[train_mask, FEATURES]
    y_train = ventas.loc[train_mask, TARGET].values.ravel()
    X_val = ventas.loc[val_mask, FEATURES]
    y_val = ventas.loc[val_mask, TARGET].values.ravel()
    
    # Train model with your custom r2_score_lgb
    reg_lgb = lgb.LGBMRegressor(
        n_estimators=600,
        learning_rate=0.3,
        max_depth=8,
        objective='regression',
        verbose=-1  # Silent mode
    )
    
    reg_lgb.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric=r2_score_lgb,  # Using your pre-defined metric
        callbacks=[
            lgb.early_stopping(stopping_rounds=early_stopping_rounds, verbose=False),
            lgb.log_evaluation(0)  # Disable logging if verbose=-1
        ]
    )
    
    # Predict and store R2 score
    y_pred = reg_lgb.predict(X_val)
    all_scores.append(r2_score(y_val, y_pred))

# Final results
print(f"Average R² over {len(all_scores)} folds: {np.mean(all_scores):.4f}")
print(f"Std R²: {np.std(all_scores):.4f}")

all_scores
"""

'   \n# Parameters\ninitial_train_weeks = 146 \nhorizon_weeks = 1           \nearly_stopping_rounds = 50  \n\n# Store results\nall_scores = []\n\n# Pre-calculate week-year identifiers (optimized)\nventas[\'week_year\'] = (ventas[\'date\'].dt.isocalendar().year * 100 + \n                       ventas[\'date\'].dt.isocalendar().week)\nall_weeks = sorted(ventas[\'week_year\'].unique())\n\n# Ensure data is sorted by date to prevent leakage\nventas = ventas.sort_values(\'date\')\n\n# Rolling window validation\nfor i in range(initial_train_weeks, len(all_weeks) - horizon_weeks + 1):\n    print(i)\n    train_weeks = all_weeks[:i]\n    val_weeks = all_weeks[i:i + horizon_weeks]\n    \n    # Create masks using the pre-calculated week_year\n    train_mask = ventas[\'week_year\'].isin(train_weeks)\n    val_mask = ventas[\'week_year\'].isin(val_weeks)\n    \n    # Data splitting\n    X_train = ventas.loc[train_mask, FEATURES]\n    y_train = ventas.loc[train_mask, TARGET].values.ravel()\n    X_val 

In [9]:
reg_lgb = lgb.LGBMRegressor(
    n_estimators=600, #deberia parar con el early stopping
    learning_rate=0.3, 
    max_depth=8,
    objective='regression',
    # colsample_bytree=0.9,
    verbose=1  
)

reg_lgb.fit(
    X_train, y_train,
    eval_set=[(X_train_interno, y_train_interno)], 
    eval_metric=r2_score_lgb,
    callbacks=[lgb.early_stopping(stopping_rounds=50),lgb.log_evaluation(50) ]
)
                    

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.308052 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2837
[LightGBM] [Info] Number of data points in the train set: 5206793, number of used features: 15
[LightGBM] [Info] Start training from score 5.233525
Training until validation scores don't improve for 50 rounds
[50]	valid_0's l2: 3.26642	valid_0's r2: 0.777459
[100]	valid_0's l2: 3.05647	valid_0's r2: 0.791763
[150]	valid_0's l2: 2.95886	valid_0's r2: 0.798413
[200]	valid_0's l2: 2.89475	valid_0's r2: 0.802781
[250]	valid_0's l2: 2.85001	valid_0's r2: 0.805829
[300]	valid_0's l2: 2.81239	valid_0's r2: 0.808392
[350]	valid_0's l2: 2.78306	valid_0's r2: 0.81039
[400]	valid_0's l2: 2.75655	valid_0's r2: 0.812196
[450]	valid_0's l2: 2.73369	valid_0's r2: 0.813754
[500]	valid_0's l2: 2.71037	valid_0's r2: 0.815343
[550]	valid_0's l2: 2.69029	valid_0's r2: 0.816711
[600]	valid_0's l2: 2.67167	valid_0'

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,8
,learning_rate,0.3
,n_estimators,600
,subsample_for_bin,200000
,objective,'regression'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [10]:
feature_importance = pd.DataFrame(
    reg_lgb.feature_importances_ / reg_lgb.feature_importances_.sum(),  # Normalize
    columns=["importance"],
    index=reg_lgb.feature_name_
).sort_values(by="importance", ascending=False) 
# Convert to percentage

feature_importance

,importance
min_diff_factory,0.131056
std_diff_factory,0.130278
max_diff_factory,0.124611
max_discount,0.112389
std_discount,0.111556
min_discount,0.104389
demand_rolling_mean_14d,0.068278
anual_sin_1,0.047611
max_price,0.039722
min_price,0.039556


In [11]:
prediccion=reg_lgb.predict(X_test_real)
kaggle=crear_archivo_kaggle(prediccion,test)

In [14]:
kaggle.describe()

,TOTAL SALES
count,77700.000000
mean,310.166278
std,424.156556
min,0.000000
25%,87.378237
50%,170.949065
75%,331.513329
max,5910.974654


In [15]:
y_test_interno.describe()

,demand
count,77672.000000
mean,4.548282
std,3.175569
min,1.000000
25%,2.000000
50%,4.000000
75%,5.999970
max,47.987219
